In [3]:
# Importar as bibliotecas
import pandas as pd
from mlxtend.frequent_patterns import apriori, association_rules
import warnings
#Como imprimia vários warnings e não tivemos tempo suficiente, solucionamos utilizamos o supressor de Warnings :)
warnings.filterwarnings('ignore', category=DeprecationWarning, module='jupyter_client')
print("Deprecation warnings from jupyter_client suppressed.")

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packag

Deprecation warnings from jupyter_client suppressed.


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


In [4]:
#Como não foi indicado um dataset, criamos o nosso próprio dataset
dados = [
    ['leite', 'pão', 'manteiga'],
    ['cerveja', 'pão', 'fralda'],
    ['leite', 'pão', 'fralda', 'cerveja'],
    ['leite', 'pão', 'manteiga', 'cerveja'],
    ['pão', 'manteiga'],
    ['leite', 'fralda'],
    ['cerveja', 'pão'],
    ['leite', 'pão', 'fralda', 'manteiga'],
]

#Converter o dataset em um DataFrame e no formato de carrinhos
from mlxtend.preprocessing import TransactionEncoder
te = TransactionEncoder()
te_ary = te.fit(dados).transform(dados)
df = pd.DataFrame(te_ary, columns=te.columns_)

print("5 primeiras linhas do dataset")
print(df.head())

5 primeiras linhas do dataset
   cerveja  fralda  leite  manteiga   pão
0    False   False   True      True  True
1     True    True  False     False  True
2     True    True   True     False  True
3     True   False   True      True  True
4    False   False  False      True  True


In [6]:
#Utilizando o algoritmo Apriori, o qual procura por combinações frequentes dos produtos comprados juntos
#O 'min_support' permite definir a frequência mínima da ocorrência de itens comprados juntos
frequent_itemsets = apriori(df, min_support=0.3, use_colnames=True)
print("Conjuntos frequentes de itens ===")
print(frequent_itemsets)


=== Conjuntos frequentes de itens ===
    support                itemsets
0     0.500               (cerveja)
1     0.500                (fralda)
2     0.625                 (leite)
3     0.500              (manteiga)
4     0.875                   (pão)
5     0.500          (cerveja, pão)
6     0.375         (fralda, leite)
7     0.375           (fralda, pão)
8     0.375       (leite, manteiga)
9     0.500            (leite, pão)
10    0.500         (pão, manteiga)
11    0.375  (leite, pão, manteiga)


In [8]:
#Para gerar as regras de associações, utilizando suporte, confiança e o lift.
regras = association_rules(frequent_itemsets, metric="lift", min_threshold=1)
regras = regras.sort_values(by='lift', ascending=False)

print("Regras de Associação")
print(regras[['antecedents', 'consequents', 'support', 'confidence', 'lift']])


Regras de Associação
          antecedents        consequents  support  confidence      lift
13         (manteiga)       (leite, pão)    0.375    0.750000  1.500000
8        (leite, pão)         (manteiga)    0.375    0.750000  1.500000
4             (leite)         (manteiga)    0.375    0.600000  1.200000
5          (manteiga)            (leite)    0.375    0.750000  1.200000
3             (leite)           (fralda)    0.375    0.600000  1.200000
2            (fralda)            (leite)    0.375    0.750000  1.200000
10    (pão, manteiga)            (leite)    0.375    0.750000  1.200000
11            (leite)    (pão, manteiga)    0.375    0.600000  1.200000
0           (cerveja)              (pão)    0.500    1.000000  1.142857
1               (pão)          (cerveja)    0.500    0.571429  1.142857
9   (leite, manteiga)              (pão)    0.375    1.000000  1.142857
7          (manteiga)              (pão)    0.500    1.000000  1.142857
6               (pão)         (manteiga)   

In [10]:
# Pra mostrar as 5 regras com mais relevância
print("5 Regras Mais Relevantes")
mais_relevantes = regras.head(5)
print(mais_relevantes[['antecedents', 'consequents', 'confidence', 'lift']])

5 Regras Mais Relevantes
     antecedents   consequents  confidence  lift
13    (manteiga)  (leite, pão)        0.75   1.5
8   (leite, pão)    (manteiga)        0.75   1.5
4        (leite)    (manteiga)        0.60   1.2
5     (manteiga)       (leite)        0.75   1.2
3        (leite)      (fralda)        0.60   1.2


In [15]:
# Recomendações: cliente compra a manteiga e leite, o algoritmo dá recomendações dos itens que ele poderia querer
itens_cliente = {'manteiga', 'leite'}

recomendacoes = []
for _, row in regras.iterrows():
    if row['antecedents'].issubset(itens_cliente):
        recomendacoes.extend(list(row['consequents']))

recomendacoes = list(set(recomendacoes) - itens_cliente)

print("O cliente compra manteiga e leite e tem como recomendação comprar:")
print(recomendacoes if recomendacoes else "Nenhuma recomendação encontrada.")


O cliente compra manteiga e leite e tem como recomendação comprar:
['fralda', 'pão']


In [17]:
""" Resposta (Continuação) As regras com maior confiança e lift inferem os produtos que tendem a ser comprados juntamente.
Utilizando essas projeções um supermercado pode reorganizar as prateleiras dos produtos, criar promoções combinadas, permitindo uma
melhor experiência para o cliente e um possível aumento das vendas por parte do mercado.
"""


' Resposta (Continuação) As regras com maior confiança e lift inferem os produtos que tendem a ser comprados juntamente.\nUtilizando essas projeções um supermercado pode reorganizar as prateleiras dos produtos, criar promoções combinadas, permitindo uma\nmelhor experiência para o cliente e um possível aumento das vendas por parte do mercado.\n'